<a href="https://colab.research.google.com/github/lapshinaaa/applied-statistics-tasks/blob/main/AS6_Bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://raw.githubusercontent.com/FUlyankin/r_probability/master/end_seminars_2020/sem08/real_expect.png" width="500">

# Bootstrap
</center>

In [ ]:
import numpy as np
import pandas as pd

from scipy import stats as sts

import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

## Task 1 (discrimination)

## 📖 Case Description: Verizon Repair Time Investigation  

**Verizon** is the primary regional telecommunications provider in the western United States, responsible for maintaining the local landline infrastructure (known as the *Incumbent Local Exchange Carrier*, or **ILEC**).  

In addition to its own services, Verizon also offered long-distance communication lines — just like several of its competitors, collectively referred to as *Competing Local Exchange Carriers* (**CLECs**).  

Whenever a malfunction occurred, Verizon was obligated to repair not only its own equipment but also that of its competitors. Crucially, it had to do so **at the same speed** — repair times for CLEC equipment were legally required to be no slower than for Verizon’s own infrastructure. The **Public Utilities Commission** monitored compliance with this rule.  

Over time, **suspicions arose** that Verizon was repairing its competitors’ equipment **more slowly** than its own. During regulatory hearings concerning these potential violations, **Tim Hesterberg**, the author of a well-known paper on the bootstrap, served as a consultant for Verizon.  

The Commission compared the repair times for Verizon’s own equipment versus CLEC equipment. For each case, they tested the **null hypothesis of equal mean repair times**, using a **significance level of 1%**.  

If more than 1% of these tests turned out significant, Verizon would face **heavy penalties**. The Commission initially relied on **t-tests** for comparing means, but following Hesterberg’s advice, Verizon proposed using **robust, nonparametric tests** — methods less sensitive to outliers and not reliant on the normality of the underlying distributions.  

In this assignment, we will approach the same question — whether Verizon systematically discriminated against competitors — using a **bootstrap approach**.  
We will:
- Construct a **confidence interval for the difference in medians**,  
- Check whether **zero lies within that interval**,  
- And use a **significance level of 1%**.  

The analysis will be done for **one CLEC company** and **Verizon**, simulating the original investigation.

In [ ]:
df = pd.read_csv('verizon.txt', sep='\t')
print(df.shape)

# ILEC - Verizion
# CLEC - внешние клиенты
df.head()

Посмотрим на то, как распределено время ремонта для разных компаний, а также на число наблюдений, которым мы располагаем.

In [ ]:
df.groupby('Group')['Time'].agg(['count', 'mean', 'median', 'std'])

В данных есть существенный перекос в количестве наблюдений. Среднее время ремонта CLEC почти вдвое больше, чем для ILEC. Вопрос в том, является ли это свидетельством дискриминации. Если посмотреть на распределение, можно заметить, что распределение времени на ремонт CLEC имеет как минимум один выброс. Тем не менее, если смотреть на медианы, которые робастны (нечувствительны) к выбросам, ситуация, связанная со временем оказывается ещё более драмматичной.

In [ ]:
plt.figure(figsize=(12,5))
df[df.Group == 'ILEC'].Time.hist(bins=15, label='Verizion', alpha=0.4, density=True)
df[df.Group == 'CLEC'].Time.hist(bins=15, label='CLEC', alpha=0.4, density=True)
plt.legend();

__а) [2 балла]__ Постройте для разницы $med_{CLEC} - med_{Verizion}$ доверительный интервал Эфрона (перцентильный доверительный интервал).

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

B_sample=10**4
left = ...
right = ...

# your code here


In [ ]:
# проверка того, что задание решено верно
assert left > 1.6
assert right < 20

# подобные тесты скрыты от вас

В целом, перцентильный интервал выглядит самым простым вариантом оценивания, однако в реальности он даёт весьма плохие результаты, так как сильно зависит от того насколько хорошо изначальная выборка аппроксимирует истинное распределение случайной величины. В данной ситуации из-за того, что исходное распределение асимметрично, мы получаем смещение.

__б) [2 балла]__ Постройте для разницы $med_{CLEC} - med_{Verizion}$ доверительный интервал Холла (обратный перцентильный доверительный интервал).

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

left = ...
right = ...

# your code here


In [ ]:
# проверка того, что задание решено верно
assert left > 3
assert right < 22

# подобные тесты скрыты от вас

Холл в своей работе называл перцентильный интервал "the wrong pivot, backwards". Обратный перцентильный интервал для асимметричных распределений делает то же самое, но в обратном направлении.

__в) [2 балла]__ Постройте $t-$ перцентильный доверительный интервал:

1. Найдите точечную оценку неизвестного параметра (разность медиан) и точечную оценку стандартного отклонения
2. Сгенерируйте $10^5$ boostrap-выборок
3. Рассчитайте по каждой выбооке выборочную разницу медиан
4. Рассчитайте по каждой выборке стандартную ошибку
5. Найдите по каждой выборке $$t^* = \frac{\hat \theta^{*} - \hat{\theta}}{S^{*}}$$
6. Найдите для $t^{*}$ квантили уровня $0.05$ и $0.95$
7. Постройте доверительный интервал для разности медиан

В качестве $S^{*}$ исполизуйте сумму стандартных отклонений, посчитанных по выборкам $x$ и $y$. Конечно же, мы используем при таком подходе неправильное стандартное отклонение. При этом мы надеемся, что бутсрэп сам поправит нашу ошибку. При таком подходе главное, чтобы знаменатель сходился по вероятности к какому-нибудь пределу.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

theta_hat = ...
std_hat = ...

left = ...
right = ...

alpha = 0.01
# your code here


In [ ]:
# проверка того, что задание решено верно
assert left < 2
assert right > 20

# подобные тесты скрыты от вас

У $t$-bootstrap есть одно интересное свойство: он "уважает" асимметрию распределений при расчёте доверительных интервалов.

__г) [2 балла]__ Что в итоге? Есть ли дискриминация на рынке?

__Ответ:__

__д) [2 балла]__ Попробуйте оценить  $S^{*}$ с помощью двойного бутстрапа:

1. Бутстрапируем $B$ выборок
2. Для каждой сбутстрапированной выборки делаем ещё $B$ итераций бутстрапа, рассматривая эту выборку как исходную
3. По каждой выборке второго уровня находим асимметрию, подсчитываем по полученным асимметриям стандартное отклонение
4. Теперь у нас есть $B$ асимметрий и $B$ их стандартных отклонений
5. Стандартное отклонение, найденное по выборке асимметрий первого уровня мы будем использовать для подсчёта наблюдаемого значения статистики

Получается матрёшка из бутстрапов. Внутри каждого бутстрапа ещё $B$ бутсрапов. Изменились ли ваши выводы?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you